<font color=Green>
    
# **4.5 Presentación del análisis**

</font>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_crudo = pd.read_csv(r'C:\Users\ldavidr\Desktop\dow_jones_index.data')
df_crudo

### El parámetro _converters_ de la función `pd.read_csv()` nos permite aplicar una operación en las columnas.

In [ ]:
columnas = ['accion', 'fecha', 'precio']

df = pd.read_csv(r'C:\Users\ldavidr\Desktop\dow_jones_index.data',
                 usecols=[1,2,6],
                 header=0,
                 names = columnas,
                 parse_dates=['fecha'],
                 converters={'precio':
                             lambda x: 
                             float(x.replace('$', ''))
                            }
                )
df

### Empecemos por entender los datos contenidos en el csv.

## <font color=#003d5c> _¿Qué período en el tiempo representan los datos?_ </font>

In [ ]:
df.fecha.dt.year.value_counts()

+ Las datos pertenecen al año 2011.

In [ ]:
df.fecha.dt.month.value_counts()

In [ ]:
df.fecha.dt.month_name().value_counts()

+ Los datos se distribuyen desde Enero a Junio del 2011: _Primer semestre del año._
+ En el mes de Abril hay 30 registros más que en el resto de los meses: _¿Por qué?._

In [ ]:
df['mes'] = df.fecha.dt.month_name()

filtro1 = df['mes'] == 'April'

filtro2 = df['mes'] == 'May'

In [ ]:
df[filtro1].groupby('accion')['mes'].value_counts()

In [ ]:
df[filtro2].groupby('accion')['mes'].value_counts()

+ En Abril las acciones cotizaron 5 veces porque el mes tuvo 5 semanas.

## <font color=#003d5c> _¿Cuáles acciones en promedio tuvieron la mayor y la menor cotización?_ </font>

In [ ]:
df.groupby('accion').agg({'precio': [np.mean, np.std]})

In [ ]:
df.groupby('accion').agg({'precio': [np.mean, np.std]}).sort_values([('precio', 'mean')], ascending=False)

+ En promedio IBM tuvo el mayor precio: 163,10 USD
+ En promedio BAC tuvo el menor precio: 13,05 USD

## <font color=#003d5c> _¿Cuáles acciones tuvieron la mayor y menor variación de precio?_ </font>

In [ ]:
df.groupby('accion').agg({'precio': [np.mean, np.std]}).sort_values([('precio', 'std')], ascending=False)

+ CAT es la acción más volátil, con una desviación tipica de 6,22 USD
+ AA es la acción más consistente, con una desviación típica de 0,77 USD

### Ya que conocemos la acción más volátil, podemos preguntarnos si el precio de esta acción tiene correlación con el precio de otras acciones.

### Deseamos estudiar una o varias acciones particulares pero el conjunto de datos no presenta un formato conveniente para esto. 

### La función `df.pivot_table()` nos genera un nuevo `DataFrame` reorganizado con la forma que deseemos.

In [ ]:
df_nuevo = df.pivot_table(index='fecha', 
                          columns='accion', 
                          values='precio')
df_nuevo

### Ahora podemos centrarnos en estudiar el comportamiento en el tiempo de cada acción.

In [ ]:
fig, (pan1,pan2) = plt.subplots(2,1, sharex=True, constrained_layout=True)

pan1.plot(df_nuevo.index, df_nuevo.CAT, 
         color = 'blue', 
         marker = 'o', 
         linestyle = 'solid',
         label = 'CAT')


pan2.plot(df_nuevo.index, df_nuevo.AA, 
          color = 'orange', 
          marker = 'v', 
          linestyle = 'solid',
          label='AA')


for pan in fig.get_axes():
    pan.set(xlabel='Fecha', 
            ylabel='Dólares americanos.', 
            title='Precio de la acción (al cierre).')

pan1.legend()
pan2.legend()

+ Los precios de ambas acciones presentan un comportamiento alcista hasta Abril -con un ligero declive en las primeras semanas de Marzo-, a partir de ahí el precio baja hasta la última semana de Junio.

## <font color=#003d5c> _¿Existe alguna correlación entre los precios de CAT y el resto de las acciones?_ </font>

In [ ]:
df_nuevo.corr()

In [ ]:
df_nuevo.corr().loc['CAT']

In [ ]:
df_nuevo.corr().loc['CAT'][df_nuevo.corr().loc['CAT'] > 0.8]

In [ ]:
corr_pos = df_nuevo.corr().loc['CAT'][df_nuevo.corr().loc['CAT'] > 0.8].sort_values(ascending=False)
corr_pos

+ Las acciones DD, VZ, CVX, MMM, XOM presentan una fuerte correlación positiva con CAT.

In [ ]:
valores = corr_pos[1:5]

plt.bar(valores.index, valores.values, color='forestgreen')

plt.plot()

plt.xlabel("Acciones")

plt.ylabel("Coeficiente de correlación.")

plt.title('Correlación entre CAT y el resto.')

plt.show()

In [ ]:
corr_neg = df_nuevo.corr().loc['CAT'][df_nuevo.corr().loc['CAT'] < 0].sort_values(ascending=True)
corr_neg

In [ ]:
valores = corr_neg[1:5]

plt.bar(valores.index, valores.values, color='brown')

plt.plot()

plt.xlabel("Acciones")

plt.ylabel("Coeficiente de correlación.")

plt.title('Correlación entre CAT y el resto.')

plt.show()

+ Ninguna acción presenta una correlación negativa significante con CAT.